# import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NU M_THREADS"] = "2"
os.environ["OMP_NUM_THREADS"] = "2"
import time

import numpy
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.cuda.amp import autocast

from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter

In [15]:
from DCT.util.dataset_loader import load_dataset
from DCT.util.optimizer_loader import load_optimizer
from DCT.util.callbacks_loader import load_callbacks
from DCT.util.model_loader import load_model
from DCT.util.preprocess_loader import load_preprocesser
from DCT.util.checkpoint import CheckPoint
from DCT.util.utils import reset_seed, get_datetime, track_maximum

from metric_utils.metrics import CategoricalAccuracy, FScore, ContinueAverage

# Arguments

In [10]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--from_config", default="", type=str)
parser.add_argument("-d", "--dataset_root", default="../datasets", type=str)
parser.add_argument("-D", "--dataset", default="SpeechCommand", type=str, help="available [ubs8k | cifar10]")

group_t = parser.add_argument_group("Commun parameters")
group_t.add_argument("-m", "--model", default="cnn03", type=str)
group_t.add_argument("--supervised_ratio", default=1.0, type=float)
group_t.add_argument("--batch_size", default=256, type=int)
group_t.add_argument("--nb_epoch", default=200, type=int)
group_t.add_argument("--learning_rate", default=0.001, type=float)
group_t.add_argument("--resume", action="store_true", default=False)
group_t.add_argument("--seed", default=1234, type=int)

group_m = parser.add_argument_group("Model parameters")
group_m.add_argument("--num_classes", default=50, type=int)

group_u = parser.add_argument_group("Datasets parameters")
group_u.add_argument("-t", "--train_folds", nargs="+", default=[1, 2, 3, 4], type=int)
group_u.add_argument("-v", "--val_folds", nargs="+", default=[5], type=int)

group_l = parser.add_argument_group("Logs")
group_l.add_argument("--checkpoint_root", default="../model_save/", type=str)
group_l.add_argument("--tensorboard_root", default="../tensorboard/", type=str)
group_l.add_argument("--checkpoint_path", default="supervised", type=str)
group_l.add_argument("--tensorboard_path", default="supervised", type=str)
group_l.add_argument("--tensorboard_sufix", default="", type=str)

args = parser.parse_args("")

tensorboard_path = os.path.join(args.tensorboard_root, args.dataset, args.tensorboard_path)
checkpoint_path = os.path.join(args.checkpoint_root, args.dataset, args.checkpoint_path)

In [11]:
args

Namespace(batch_size=256, checkpoint_path='supervised', checkpoint_root='../model_save/', dataset='SpeechCommand', dataset_root='../datasets', from_config='', learning_rate=0.001, model='cnn03', nb_epoch=200, num_classes=50, resume=False, seed=1234, supervised_ratio=1.0, tensorboard_path='supervised', tensorboard_root='../tensorboard/', tensorboard_sufix='', train_folds=[1, 2, 3, 4], val_folds=[5])

# initialisation

In [12]:
reset_seed(args.seed)


# Prepare the dataset

In [13]:
args.dataset_root

'../datasets'

In [21]:
train_transform, val_transform = load_preprocesser(args.dataset, "supervised")

In [25]:
manager, train_loader, val_loader = load_dataset(
    args.dataset,
    "supervised",
    
    dataset_root = args.dataset_root,
    supervised_ratio = args.supervised_ratio,
    batch_size = args.batch_size,
    train_folds = args.train_folds,
    val_folds = args.val_folds,

    train_transform=train_transform,
    val_transform=val_transform,

    verbose = 2
)

../datasets/SpeechCommands/speech_commands_v0.02
Dataset already download and verified


# Prep model

In [29]:
torch.cuda.empty_cache()

model_func = load_model(args.dataset, args.model)
# model_func = get_model_from_name("esc_wideresnet28_8")
model = model_func(input_shape=(64, 32), num_classes = args.num_classes)
model = model.cuda()


144 50


In [30]:
from torchsummaryX import summary
input_tensor = torch.zeros((1, 64, 32), dtype=torch.float)
input_tensor = input_tensor.cuda()

s = summary(model, input_tensor)


                               Kernel Shape     Output Shape   Params  \
Layer                                                                   
0_features.0.Conv2d_0         [1, 24, 3, 3]  [1, 24, 64, 32]    240.0   
1_features.0.MaxPool2d_1                  -  [1, 24, 16, 16]        -   
2_features.0.BatchNorm2d_2             [24]  [1, 24, 16, 16]     48.0   
3_features.0.ReLU6_3                      -  [1, 24, 16, 16]        -   
4_features.1.Conv2d_0        [24, 48, 3, 3]  [1, 48, 16, 16]  10.416k   
5_features.1.MaxPool2d_1                  -    [1, 48, 4, 8]        -   
6_features.1.BatchNorm2d_2             [48]    [1, 48, 4, 8]     96.0   
7_features.1.ReLU6_3                      -    [1, 48, 4, 8]        -   
8_features.2.Conv2d_0        [48, 72, 3, 3]    [1, 72, 4, 8]  31.176k   
9_features.2.MaxPool2d_1                  -    [1, 72, 2, 4]        -   
10_features.2.BatchNorm2d_2            [72]    [1, 72, 2, 4]    144.0   
11_features.2.ReLU6_3                     -    [1, 

## Prep training

In [31]:
nb_conv = 0

for layer in s.index.values:
    if "Conv" in layer:
        nb_conv += 1
print(nb_conv)

5


# training parameters

In [32]:
# tensorboard
title_element = (args.model, args.supervised_ratio, get_datetime(), model_func.__name__, args.supervised_ratio)
tensorboard_title = "%s/%sS/%s_%s_%.1fS" % title_element

title_element = (model_func.__name__, args.supervised_ratio)
checkpoint_title = "%s_%.1fS" % title_element

tensorboard = SummaryWriter(log_dir="%s/%s" % (tensorboard_path, tensorboard_title), comment=model_func.__name__)
print(os.path.join(tensorboard_path, tensorboard_title))

# losses
loss_ce = nn.CrossEntropyLoss(reduction="mean")

../tensorboard/SpeechCommand/supervised/cnn03/1.0S/2020-09-22_16:41:22_cnn03_1.0S


In [33]:
tensorboard_params = {}
for key, value in args.__dict__.items():
    tensorboard_params[key] = str(value)

In [34]:
tensorboard.add_hparams(tensorboard_params, {})

## optimizer & callbacks

In [37]:
optimizer = load_optimizer(args.dataset, "supervised", model=model)
callbacks = load_callbacks(args.dataset, "supervised", optimizer=optimizer, nb_epoch=args.nb_epoch)


In [42]:
# Checkpoint
checkpoint = CheckPoint(model, optimizer, mode="max", name="%s/%s.torch" % (checkpoint_path, checkpoint_title))

# Metrics
fscore_fn = FScore()
acc_fn = CategoricalAccuracy()
avg = ContinueAverage()
maximum_tracker = track_maximum()

reset_metrics = lambda : [m.reset() for m in [fscore_fn, acc_fn, avg]]

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

## Can resume previous training

In [49]:
if args.resume:
    checkpoint.load_last()

In [50]:
args.resume

False

## training function

In [51]:
UNDERLINE_SEQ = "\033[1;4m"
RESET_SEQ = "\033[0m"


header_form = "{:<8.8} {:<6.6} - {:<6.6} - {:<8.8} {:<6.6} - {:<9.9} {:<12.12}| {:<9.9}- {:<6.6}"
value_form  = "{:<8.8} {:<6} - {:<6} - {:<8.8} {:<6.4f} - {:<9.9} {:<10.4f}| {:<9.4f}- {:<6.4f}"

header = header_form.format(
    ".               ", "Epoch", "%", "Losses:", "ce", "metrics: ", "acc", "F1 ","Time"
)


train_form = value_form
val_form = UNDERLINE_SEQ + value_form + RESET_SEQ

print(header)

.        Epoch  - %      - Losses:  ce     - metrics:  acc         | F1       - Time  


In [26]:
def train(epoch):
    start_time = time.time()
    print("")

    reset_metrics()
    model.train()

    for i, (X, y) in enumerate(train_loader):        
        X = X.cuda()
        y = y.cuda()
        
        logits = model(X)        
        loss = loss_ce(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.set_grad_enabled(False):
            pred = torch.softmax(logits, dim=1)
            pred_arg = torch.argmax(logits, dim=1)
            y_one_hot = F.one_hot(y, num_classes=args.num_classes)

            acc = acc_fn(pred_arg, y).mean
            fscore = fscore_fn(pred, y_one_hot).mean
            avg_ce = avg(loss.item()).mean

            # logs
            print(train_form.format(
                "Training: ",
                epoch + 1,
                int(100 * (i + 1) / len(train_loader)),
                "", avg_ce,
                "", acc, fscore,
                time.time() - start_time
            ), end="\r")

    tensorboard.add_scalar("train/Lce", avg_ce, epoch)
    tensorboard.add_scalar("train/f1", fscore, epoch)
    tensorboard.add_scalar("train/acc", acc, epoch)

In [27]:
def val(epoch):
    start_time = time.time()
    print("")
    reset_metrics()
    model.eval()

    with torch.set_grad_enabled(False):
        for i, (X, y) in enumerate(val_loader):
            X = X.cuda()
            y = y.cuda()

            logits = model(X)
            loss = loss_ce(logits, y)

            # metrics
            pred = torch.softmax(logits, dim=1)
            pred_arg = torch.argmax(logits, dim=1)
            y_one_hot = F.one_hot(y, num_classes=args.num_classes)

            acc = acc_fn(pred_arg, y).mean
            fscore = fscore_fn(pred, y_one_hot).mean
            avg_ce = avg(loss.item()).mean

            # logs
            print(val_form.format(
                "Validation: ",
                epoch + 1,
                int(100 * (i + 1) / len(val_loader)),
                "", avg_ce,
                "", acc, fscore,
                time.time() - start_time
            ), end="\r")

    tensorboard.add_scalar("val/Lce", avg_ce, epoch)
    tensorboard.add_scalar("val/f1", fscore, epoch)
    tensorboard.add_scalar("val/acc", acc, epoch)
    
    tensorboard.add_scalar("hyperparameters/learning_rate", get_lr(optimizer), epoch)
    
    tensorboard.add_scalar("max/acc", maximum_tracker("acc", acc), epoch )
    tensorboard.add_scalar("max/f1", maximum_tracker("f1", fscore), epoch )

    checkpoint.step(acc)
    lr_scheduler.step()


In [28]:
print(header)

start_epoch = checkpoint.epoch_counter
end_epoch = args.nb_epoch

for e in range(start_epoch, args.nb_epoch):
    train(e)
    val(e)
    
    tensorboard.flush()
tensorboard.close()

.        Epoch  - %      - Losses:  ce     - metrics:  acc         | F1       - Time  

Training 1      - 100    -          1.4359 -           0.5988    | 0.5415   - 92.3807
Validati 1      - 100    -          0.6513 -           0.8065    | 0.8182   - 21.4087
Training 2      - 100    -          0.5007 -           0.8533    | 0.8597   - 5.9780
Validati 2      - 100    -          0.4471 -           0.8661    | 0.8785   - 0.5101
Training 3      - 100    -          0.3802 -           0.8881    | 0.8958   - 5.3418
Validati 3      - 100    -          0.4105 -           0.8796    | 0.8907   - 0.4340
Training 4      - 100    -          0.3187 -           0.9054    | 0.9128   - 5.5625
Validati 4      - 100    -          0.4307 -           0.8728    | 0.8861   - 0.4723
Training 5      - 100    -          0.2784 -           0.9162    | 0.9239   - 5.7993
Validati 5      - 100    -          0.3812 -           0.8885    | 0.9009   - 0.4695
Training 6      - 100    -          0.2487 -           0.925

# ♫♪.ılılıll|̲̅̅●̲̅̅|̲̅̅=̲̅̅|̲̅̅●̲̅̅|llılılı.♫♪